In [34]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os, sys, h5py, time
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import tensorflow as tf
import scipy

import sys
sys.path.append('../../..')
import mutagenesisfunctions as mf
import bpdev as bd
import helper
from deepomics import neuralnetwork as nn
from deepomics import utils, fit, visualize, saliency

import contacts

from Bio import AlignIO
import time as time
import pandas as pd
np.random.seed(42)

# Get Performance Metrics

In [42]:
metricfile = 'test_metrics.csv'
metrics = pd.read_csv(metricfile)[['Model Architecture', 'Family', 'Accuracy', 'AUC-roc']]
mlp_met = metrics[metrics['Model Architecture'] == 'mlp']
redidx = [False if 'bpm' in e else True for e in mlp_met['Family']]
red_met = mlp_met[redidx][['Family', 'Accuracy', 'AUC-roc']]
metfamidx = mlp_met.set_index('Family')[['Accuracy', 'AUC-roc']].round(3)
families = list(red_met['Family'])

In [43]:
metfamidx

,Accuracy,AUC-roc
Family,,
RF00002,1.000,1.000
RF00005,0.999,1.000
RF00010,0.999,1.000
RF00017,0.998,1.000
RF00023,0.999,1.000
RF00050,0.998,1.000
RF00059,0.999,1.000
RF00162,0.995,1.000
RF00167,0.996,1.000


In [47]:
#import length information
infofile = 'marks_20_cutinfo.csv'
infodf = pd.read_csv(infofile)
inforel = infodf[[True if infodf.loc[ii, 'RFAM_ID'] in families else False for ii in infodf.index]]

#Add accuracy from metricsdf and numbasepairs
for ii in inforel.index:
    inforel.loc[ii, 'Accuracy'] = metfamidx.loc[inforel.loc[ii,'RFAM_ID'], 'Accuracy']
    inforel.loc[ii, 'AUC-roc'] = metfamidx.loc[inforel.loc[ii,'RFAM_ID'], 'AUC-roc']
    

/Users/steffanpaul/anaconda/envs/tensorflow/lib/python3.6/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Users/steffanpaul/anaconda/envs/tensorflow/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [48]:
inforel = inforel.set_index('RFAM_ID')
#inforel.to_csv('table1.csv')

In [49]:
inforel

,Name,Length,M,M_eff,Accuracy,AUC-roc
RFAM_ID,,,,,,
RF00002,5.8S ribosomal RNA,155,375612,1197.3,1.000,1.000
RF00005,tRNA,73,298470,3533.2,0.999,1.000
RF00010,Bacterial RNase P class A,359,6397,1215.1,0.999,1.000
RF00017,Metazoan signal recognition particle RNA,300,22685,4358.5,0.998,1.000
RF00023,transfer-messenger RNA,366,5983,1558.9,0.999,1.000
RF00050,FMN riboswitch (RFN element),135,4516,367.1,0.998,1.000
RF00059,TPP riboswitch (THI element),105,11197,2249.5,0.999,1.000
RF00162,SAM riboswitch (S box leader),108,4757,585.7,0.995,1.000
RF00167,Purine riboswitch,103,2427,471.1,0.996,1.000
